In [1]:
import pandas as pd
import numpy as np

In [100]:
clinical_df = pd.read_csv('data/clinical.txt',sep='\t').set_index('sampleID')
clinical_columns = [col for col in clinical_df.columns if not col.startswith('_')]
clinical_df = clinical_df.loc[:,clinical_columns]
gene_df = pd.read_csv('data/gene_expression_array.txt',sep='\t').set_index('sample').T
original_gene_df = pd.read_csv('data/gene_expression_array.txt',sep='\t').set_index('sample').T
original_gene_df.index.name = 'sampleID'
gene_df = gene_df - gene_df.mean(axis=0)
normal_samples = clinical_df.loc[clinical_df['sample_type'] == 'Solid Tissue Normal'].index.intersection(gene_df.index)
gene_df.index.name = 'sampleID'
normal_gene_df = gene_df.loc[normal_samples]
tumor_gene_df = gene_df[gene_df.index.map(lambda x:x not in normal_samples)]

In [103]:
clinical_df.index = clinical_df.index.map(lambda x:'-'.join(x.split('-')[:-1]))
tumor_gene_df.index = tumor_gene_df.index.map(lambda x:'-'.join(x.split('-')[:-1]))
original_gene_df.index = original_gene_df.index.map(lambda x:'-'.join(x.split('-')[:-1]))
patient_id_with_image = pd.read_csv('patient_id',header=None)[0]
intersected_index = tumor_gene_df.index.intersection(patient_id_with_image)
all_df = clinical_df[['clinical_stage']].loc[intersected_index].join(tumor_gene_df,how='inner').dropna()

In [63]:
# rna_seq_df = pd.read_csv('data/TCGA-OV.htseq_fpkm.tsv',sep='\t').set_index('Ensembl_ID').T
# rna_seq_df.index.name = 'sampleID'
# rna_seq_df.index = rna_seq_df.index.map(lambda x:'-'.join(x.split('-')[:-1]))

In [74]:
import scipy.stats as stats

In [76]:
rows = []
for gene in gene_df.columns:
    normal = normal_gene_df[gene]
    tumor = all_df[gene]
    res = stats.ttest_ind(normal, tumor)
    rows.append([gene,res.pvalue])

In [77]:
p_val_df = pd.DataFrame(rows,columns =['Gene','pvalue'])
p_val_df = p_val_df.sort_values(by='pvalue')

In [91]:
gene_list = p_val_df.loc[p_val_df['pvalue']<0.05,'Gene'].values

In [87]:
p_val_df[p_val_df['Gene'] == 'BRCA2']

,Gene,pvalue
1950,BRCA2,0.00484


In [80]:
p_val_df.loc[p_val_df['pvalue']<0.05,'Gene'].to_csv('gene_list.txt',index=False,header=False)

In [95]:
all_df[['clinical_stage','BRCA2']].to_csv('BRCA2_expression.tsv',sep='\t')

In [96]:
all_df[['clinical_stage']+list(gene_list)].to_csv('all_DEG_expression.tsv',sep='\t')

In [105]:
all_df[['clinical_stage','BRCA2']].join(original_gene_df['BRCA2'],how='inner',lsuffix='_zero_centered',rsuffix='_original').to_csv('BRCA2_expression.tsv',sep='\t')